In [ ]:
# Must be set before importing mujoco
import os
os.environ["MUJOCO_GL"] = "egl"

In [ ]:
# --- Install prerequisites ---
!pip install mujoco
!pip install mujoco_mjx
!pip install brax
!pip install mediapy
!pip install ml_collections

# Install MuJoCo Playground from GitHub
!pip install git+https://github.com/google-deepmind/mujoco_playground.git

# Ensure ffmpeg exists
!command -v ffmpeg >/dev/null || (apt update && apt install -y ffmpeg)

# Disable Warp
import os
os.environ["MJX_ENABLE_WARP"] = "0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.5/243.5 kB 17.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 736.6/736.6 kB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 344.1/344.1 kB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.4/172.4 kB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.7/76.7 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00
  Cloning https://github.com/google-deepmind/mujoco_playground.git to /tmp/pip-req-build-76jsyc6q
  Running command git clone --filter=blob:none --quiet https://github.com/google-deepmind/mujoco_playground.git 

In [ ]:
# Plotting + utilities
import json, itertools, time
import numpy as np
np.set_printoptions(precision=3, suppress=True, linewidth=100)
from math import cos, sin
import matplotlib.pyplot as plt
from scipy.optimize import minimize

import mediapy as media
import matplotlib.pyplot as plt

# Brax / MuJoCo / JAX
from datetime import datetime
import functools
import jax
from jax import numpy as jp

import mujoco
from mujoco import mjx

# Brax training
from brax import base, envs, math
from brax.base import Base, Motion, Transform
from brax.base import State as PipelineState
from brax.envs.base import Env, PipelineEnv, State
from brax.io import html, mjcf, model
from brax.mjx.base import State as MjxState
from brax.training.agents.ppo import networks as ppo_networks
from brax.training.agents.ppo import train as ppo
from brax.training.agents.sac import networks as sac_networks
from brax.training.agents.sac import train as sac

from etils import epath
from flax import struct
from flax.training import orbax_utils
from orbax import checkpoint as ocp

from IPython.display import HTML, clear_output
from ml_collections import config_dict

Failed to import warp: No module named 'warp'
Failed to import mujoco_warp: No module named 'warp'


In [ ]:
from mujoco_playground import wrapper, registry
from mujoco_playground.config import manipulation_params

env_name = 'PandaPickCubeOrientation'

env = registry.load(env_name)
env_cfg = registry.get_default_config(env_name)

ppo_params = manipulation_params.brax_ppo_config(env_name)
ppo_params

mujoco_menagerie not found. Downloading...


Cloning mujoco_menagerie: ██████████| 100/100 [00:43<00:00]


Checking out commit 1b86ece576591213e2b666ebf59508454200ca97
Successfully downloaded mujoco_menagerie


action_repeat: 1
batch_size: 512
discounting: 0.97
entropy_cost: 0.02
episode_length: 150
learning_rate: 0.001
network_factory:
  policy_hidden_layer_sizes: !!python/tuple
  - 32
  - 32
  - 32
  - 32
  policy_obs_key: state
  value_hidden_layer_sizes: !!python/tuple
  - 256
  - 256
  - 256
  - 256
  - 256
  value_obs_key: state
normalize_observations: true
num_envs: 2048
num_evals: 4
num_minibatches: 32
num_resets_per_eval: 10
num_timesteps: 20000000
num_updates_per_batch: 8
reward_scaling: 1.0
unroll_length: 10

In [ ]:
import textwrap
import tempfile

In [ ]:
import shutil

shutil.copy(
    "/content/drive/MyDrive/Colab Notebooks/Parts/base.stl",
    "base.stl"
)
shutil.copy(
    "/content/drive/MyDrive/Colab Notebooks/Parts/link1.stl",
    "link1.stl"
)
shutil.copy(
    "/content/drive/MyDrive/Colab Notebooks/Parts/link2.stl",
    "link2.stl"
)

!ls -lah

total 636K
drwxr-xr-x 1 root root 4.0K Dec  6 17:50 .
drwxr-xr-x 1 root root 4.0K Dec  6 17:46 ..
-rw------- 1 root root 135K Dec  6 17:50 base.stl
drwxr-xr-x 4 root root 4.0K Nov 20 14:30 .config
drwx------ 5 root root 4.0K Dec  6 17:46 drive
-rw------- 1 root root 232K Dec  6 17:50 link1.stl
-rw------- 1 root root 248K Dec  6 17:50 link2.stl
drwxr-xr-x 1 root root 4.0K Nov 20 14:30 sample_data


In [ ]:
# MuJoCo XML (X-axis rotation for both joints)
xml = """
<mujoco>
  <option gravity="0 0 -9.81" integrator="RK4" timestep="0.002"/>

  <asset>
    <mesh name="base_mesh"  file="base.stl"  scale="0.001 0.001 0.001"/>
    <mesh name="link1_mesh" file="link1.stl" scale="0.001 0.001 0.001"/>
    <mesh name="link2_mesh" file="link2.stl" scale="0.001 0.001 0.001"/>
  </asset>

  <worldbody>

    <light name="top" pos="0 0 1"/>
    <light name="side" pos="1 0 0.5"/>

    <!-- BASE -->
    <body name="base" pos="0 0 0">
      <!-- Visual geometry only, no collision -->
      <geom type="mesh" mesh="base_mesh" rgba="0.8 0.8 0.8 1" contype="0" conaffinity="0"/>

      <!-- LINK 1 (bottom arm) -->
      <body name="link1" pos="0 0 0">
        <joint name="joint1" type="hinge" axis="0 1 0" pos="0 0 0"
               range="-170 170" damping="0.5"/>

        <!-- Inertia at link1's actual center of mass -->
        <inertial pos="-0.004529 -0.015217 0.083594" mass="1.0" diaginertia="0.01 0.01 0.01"/>

        <!-- Visual geometry only, no collision with base -->
        <geom type="mesh" mesh="link1_mesh" rgba="0.2 0.6 1 1"
              contype="1" conaffinity="1" group="0"/>

        <!-- LINK 2 (top arm) - positioned at link1's endpoint -->
        <body name="link2" pos="-0.004529 -0.015217 0.186068">
          <joint name="joint2" type="hinge" axis="0 1 0" pos="0 0 0"
                 range="-170 170" damping="0.5"/>

          <!-- Inertia at link2's actual center of mass (relative to link2 origin) -->
          <inertial pos="0.000256 0.005679 0.111" mass="0.8" diaginertia="0.01 0.01 0.008"/>

          <!-- Link2 mesh with corrected offset to align base with joint -->
          <geom type="mesh" mesh="link2_mesh" pos="0 0 -0.153697" rgba="1 0.6 0.2 1"
                contype="1" conaffinity="1" group="0"/>
        </body>

      </body>
    </body>

  </worldbody>

  <actuator>
    <!-- Increased gear ratios for better control -->
    <motor joint="joint1" ctrlrange="-10 10" gear="20"/>
    <motor joint="joint2" ctrlrange="-10 10" gear="15"/>
  </actuator>

</mujoco>
"""

# Make model, data, and renderer
mj_model = mujoco.MjModel.from_xml_string(xml)
mj_data = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model)

In [ ]:
mjx_model = mjx.put_model(mj_model)
mjx_data = mjx.put_data(mj_model, mj_data)

In [ ]:
print(mj_data.qpos, type(mj_data.qpos))
print(mjx_data.qpos, type(mjx_data.qpos), mjx_data.qpos.devices())

[0. 0.] <class 'numpy.ndarray'>
[0. 0.] <class 'jaxlib._jax.ArrayImpl'> {CpuDevice(id=0)}


In [ ]:
import numpy as np
import struct

def read_stl_file(filename):
    """Read STL file and extract vertex information"""
    try:
        with open(filename, 'rb') as f:
            # Read header (80 bytes)
            header = f.read(80)

            # Read number of triangles (4 bytes)
            num_triangles = struct.unpack('I', f.read(4))[0]

            vertices = []

            # Read each triangle
            for _ in range(num_triangles):
                # Normal vector (12 bytes) - skip
                f.read(12)

                # Read 3 vertices (3 * 12 bytes each)
                for _ in range(3):
                    vertex = struct.unpack('fff', f.read(12))
                    vertices.append(vertex)

                # Attribute byte count (2 bytes) - skip
                f.read(2)

            vertices = np.array(vertices)
            return vertices, num_triangles
    except FileNotFoundError:
        print(f"ERROR: File '{filename}' not found!")
        return None, 0
    except Exception as e:
        print(f"ERROR reading '{filename}': {e}")
        return None, 0

def analyze_stl_mesh(vertices, mesh_name):
    """Analyze STL mesh geometry"""
    if vertices is None or len(vertices) == 0:
        print(f"\n{'='*60}")
        print(f"MESH: {mesh_name} - NO DATA")
        print(f"{'='*60}")
        return

    print(f"\n{'='*60}")
    print(f"MESH: {mesh_name}")
    print(f"{'='*60}")

    # Basic stats
    print(f"Number of vertices: {len(vertices)}")
    print(f"Number of triangles: {len(vertices) // 3}")

    # Bounding box
    min_coords = vertices.min(axis=0)
    max_coords = vertices.max(axis=0)
    dimensions = max_coords - min_coords
    center = (max_coords + min_coords) / 2

    print(f"\nBounding Box (in STL units, typically mm):")
    print(f"  X: [{min_coords[0]:8.3f}, {max_coords[0]:8.3f}] → width:  {dimensions[0]:8.3f}")
    print(f"  Y: [{min_coords[1]:8.3f}, {max_coords[1]:8.3f}] → depth:  {dimensions[1]:8.3f}")
    print(f"  Z: [{min_coords[2]:8.3f}, {max_coords[2]:8.3f}] → height: {dimensions[2]:8.3f}")

    print(f"\nCenter point: [{center[0]:8.3f}, {center[1]:8.3f}, {center[2]:8.3f}]")

    # After 0.001 scaling (as in your XML)
    print(f"\nAfter 0.001 scaling (STL mm → MuJoCo m):")
    print(f"  Dimensions: [{dimensions[0]*0.001:.6f}, {dimensions[1]*0.001:.6f}, {dimensions[2]*0.001:.6f}]")
    print(f"  Center:     [{center[0]*0.001:.6f}, {center[1]*0.001:.6f}, {center[2]*0.001:.6f}]")

    # Origin offset (important for alignment)
    print(f"\nOrigin offset from geometric center:")
    print(f"  Raw: [{-center[0]:8.3f}, {-center[1]:8.3f}, {-center[2]:8.3f}] mm")
    print(f"  Scaled: [{-center[0]*0.001:.6f}, {-center[1]*0.001:.6f}, {-center[2]*0.001:.6f}] m")

    # Connection points (assuming +Z is up)
    print(f"\nLikely connection points:")
    print(f"  Bottom (min Z): [{center[0]*0.001:.6f}, {center[1]*0.001:.6f}, {min_coords[2]*0.001:.6f}]")
    print(f"  Top (max Z):    [{center[0]*0.001:.6f}, {center[1]*0.001:.6f}, {max_coords[2]*0.001:.6f}]")

    return {
        'min': min_coords,
        'max': max_coords,
        'dimensions': dimensions,
        'center': center,
        'scaled_dimensions': dimensions * 0.001,
        'scaled_center': center * 0.001
    }

# Read and analyze all three STL files
print("="*60)
print("STL FILE ANALYSIS FOR ROBOT ARM")
print("="*60)

base_vertices, base_triangles = read_stl_file('base.stl')
link1_vertices, link1_triangles = read_stl_file('link1.stl')
link2_vertices, link2_triangles = read_stl_file('link2.stl')

base_info = analyze_stl_mesh(base_vertices, 'BASE')
link1_info = analyze_stl_mesh(link1_vertices, 'LINK 1')
link2_info = analyze_stl_mesh(link2_vertices, 'LINK 2')

# Kinematic chain recommendations
if all([base_info, link1_info, link2_info]):
    print(f"\n{'='*60}")
    print("KINEMATIC CHAIN RECOMMENDATIONS")
    print(f"{'='*60}")

    print("\nCurrent XML configuration:")
    print("  base:  pos='0 0 0'")
    print("  link1: pos='0 0 0' (relative to base)")
    print("  link2: pos='0.1 0 0.20077' (relative to link1)")

    print("\n⚠️  PROBLEMS DETECTED:")
    print(f"1. Link1 height (scaled): {link1_info['scaled_dimensions'][2]:.6f} m")
    print(f"   But link2 is offset by: 0.20077 m in Z")
    print(f"   Difference: {abs(link1_info['scaled_dimensions'][2] - 0.20077):.6f} m")

    print(f"\n2. Link1 X-extent: {link1_info['scaled_dimensions'][0]:.6f} m")
    print(f"   But link2 has X offset: 0.1 m")
    print(f"   This suggests possible misalignment!")

    print("\n✓ RECOMMENDED FIXES:")

    # Where does link1 actually end?
    link1_top_z = link1_info['max'][2] * 0.001
    link1_top_x = link1_info['center'][0] * 0.001
    link1_top_y = link1_info['center'][1] * 0.001

    print(f"\nLink1 top connection point (where link2 should attach):")
    print(f"  pos='{link1_top_x:.6f} {link1_top_y:.6f} {link1_top_z:.6f}'")

    print(f"\nSuggested XML correction:")
    print(f"  <body name='link2' pos='{link1_top_x:.6f} {link1_top_y:.6f} {link1_top_z:.6f}'>")

    # Check if link2 mesh needs offset adjustment
    link2_bottom_z = link2_info['min'][2] * 0.001
    if abs(link2_bottom_z) > 0.001:
        print(f"\n⚠️  Link2 mesh origin is NOT at its base!")
        print(f"     Mesh bottom is at Z = {link2_bottom_z:.6f}")
        print(f"     You may need to adjust the geom pos in link2")
        print(f"     Current: pos='0 0 -0.04'")
        print(f"     Try: pos='0 0 {-link2_bottom_z:.6f}'")

print(f"\n{'='*60}")
print("DIAGNOSIS COMPLETE")
print(f"{'='*60}")
# Create model and data
mj_model = mujoco.MjModel.from_xml_string(xml)
mj_data = mujoco.MjData(mj_model)

print("=== KINEMATIC CHAIN DIAGNOSIS ===\n")

# Check body positions and orientations
print("Body hierarchy and positions:")
for i in range(mj_model.nbody):
    body_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_BODY, i)
    body_pos = mj_model.body_pos[i]
    parent_id = mj_model.body_parentid[i]
    parent_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_BODY, parent_id) if parent_id >= 0 else "world"
    print(f"  Body {i} ({body_name}): parent={parent_name}, pos={body_pos}")

print("\n=== JOINT DIAGNOSIS ===")
for i in range(mj_model.njnt):
    joint_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_JOINT, i)
    joint_pos = mj_model.jnt_pos[i]  # Joint position in parent body frame
    joint_axis = mj_model.jnt_axis[i]  # Joint axis
    body_id = mj_model.jnt_bodyid[i]  # Body the joint belongs to
    body_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_BODY, body_id)
    print(f"Joint {i} ({joint_name}):")
    print(f"  - Attached to body: {body_name}")
    print(f"  - Position: {joint_pos}")
    print(f"  - Axis: {joint_axis}")

print("\n=== TESTING FORWARD KINEMATICS ===")
mujoco.mj_resetData(mj_model, mj_data)

# Test different joint configurations
test_configs = [
    ("Zero position", [0, 0]),
    ("Joint1 = 90°", [np.pi/2, 0]),
    ("Joint2 = 90°", [0, np.pi/2]),
    ("Both = 45°", [np.pi/4, np.pi/4]),
]

for config_name, qpos in test_configs:
    mj_data.qpos[:] = qpos
    mujoco.mj_forward(mj_model, mj_data)

    print(f"\n{config_name}: qpos = {np.degrees(qpos)}")

    # Get global positions of bodies
    for i in range(mj_model.nbody):
        body_name = mujoco.mj_id2name(mj_model, mujoco.mjtObj.mjOBJ_BODY, i)
        if body_name in ['link1', 'link2']:
            xpos = mj_data.xpos[i]
            xmat = mj_data.xmat[i].reshape(3, 3)
            euler = np.array([
                np.arctan2(xmat[2, 1], xmat[2, 2]),
                np.arctan2(-xmat[2, 0], np.sqrt(xmat[2, 1]**2 + xmat[2, 2]**2)),
                np.arctan2(xmat[1, 0], xmat[0, 0])
            ])
            print(f"  {body_name} global pos: {xpos}, euler (deg): {np.degrees(euler)}")

STL FILE ANALYSIS FOR ROBOT ARM

MESH: BASE
Number of vertices: 8232
Number of triangles: 2744

Bounding Box (in STL units, typically mm):
  X: [ -76.528,   73.531] → width:   150.059
  Y: [ -99.012,   51.216] → depth:   150.229
  Z: [-100.516,   15.885] → height:  116.401

Center point: [  -1.498,  -23.898,  -42.316]

After 0.001 scaling (STL mm → MuJoCo m):
  Dimensions: [0.150059, 0.150229, 0.116401]
  Center:     [-0.001498, -0.023898, -0.042316]

Origin offset from geometric center:
  Raw: [   1.498,   23.898,   42.316] mm
  Scaled: [0.001498, 0.023898, 0.042316] m

Likely connection points:
  Bottom (min Z): [-0.001498, -0.023898, -0.100516]
  Top (max Z):    [-0.001498, -0.023898, 0.015885]

MESH: LINK 1
Number of vertices: 14238
Number of triangles: 4746

Bounding Box (in STL units, typically mm):
  X: [ -26.993,   17.936] → width:    44.929
  Y: [ -30.656,    0.221] → depth:    30.877
  Z: [ -18.879,  186.068] → height:  204.947

Center point: [  -4.529,  -15.217,   83.594]

A

# **Unoptimized Arm Motion**

In [ ]:
# Create model and data
mj_model = mujoco.MjModel.from_xml_string(xml)
mj_data = mujoco.MjData(mj_model)

renderer = mujoco.Renderer(mj_model, height=480, width=640)

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = 8.0
framerate = 60
frames = []

mujoco.mj_resetData(mj_model, mj_data)

# Starting pose
start_j1_deg = -10
start_j2_deg = 20

# Target pose
target_j1_deg = 80
target_j2_deg = 20

# Set initial qpos
mj_data.qpos[0] = np.radians(start_j1_deg)
mj_data.qpos[1] = np.radians(start_j2_deg)

def min_jerk(alpha):
    """Smooth interpolation 0 to 1 with zero jerk at boundaries."""
    return 10*alpha**3 - 15*alpha**4 + 6*alpha**5

while mj_data.time < duration:
    t = mj_data.time

    # 0 → 1 normalized time
    alpha = np.clip(t / duration, 0, 1)

    # Smooth interpolation
    s = min_jerk(alpha)

    # Desired angles
    desired_j1 = np.radians(start_j1_deg + s * (target_j1_deg - start_j1_deg))
    desired_j2 = np.radians(start_j2_deg + s * (target_j2_deg - start_j2_deg))

    # PD Controller
    kp = 40      # lower gains = smoother motion
    kd = 3

    mj_data.ctrl[0] = kp * (desired_j1 - mj_data.qpos[0]) - kd * mj_data.qvel[0]
    mj_data.ctrl[1] = kp * (desired_j2 - mj_data.qpos[1]) - kd * mj_data.qvel[1]

    mujoco.mj_step(mj_model, mj_data)

    # Save frame
    if len(frames) < mj_data.time * framerate:
        renderer.update_scene(mj_data, scene_option=scene_option)
        frames.append(renderer.render())

media.show_video(frames, fps=framerate)

This browser does not support the video tag.


# **Optimized Arm Motion**

In [ ]:
# Physical parameters
m1 = 1.0   # mass of link1 sliding along x
m2 = 1.0   # mass of link2 sliding along z
g_val = 9.81  # gravity magnitude (m/s^2), z points up in our FK convention

# geometry constants
a_1  = 0.3160
l_2  = 0.12

# angular start/end
# We'll convert them into prismatic targets using the decoupled kinematic mapping:
q0_angles = np.array([90, 90]) * np.pi/180.0
qf_angles = np.array([20, 80]) * np.pi/180.0

def angular_to_prismatic(qa):
    # qa = [q1_angle, q2_angle]
    x = a_1 * np.sin(qa[0])   # side-to-side
    z = l_2 * np.sin(qa[1])   # up-down
    return np.array([x, z])

q0 = angular_to_prismatic(q0_angles)   # prismatic start
qf = angular_to_prismatic(qf_angles)   # prismatic goal

# Trajectory discretization
T = 8.0
N = 100
t = np.linspace(0, T, N)
dt = t[1] - t[0]

# Reference
q_ref = np.vstack([
    np.linspace(q0[0], qf[0], N),
    np.linspace(q0[1], qf[1], N)
])

# Forward kinematics (prismatic decoupled)
def fk_position_prismatic(q):
    # q = [x, z] in world coordinates
    return np.array([q[0], q[1]])  # (x, z)

p_target = fk_position_prismatic(qf)

# Dynamics for decoupled prismatic system
def dynamics_prismatic(q, qd, qdd):
    """
    q, qd, qdd: shape (2,)
    returns tau: shape (2,)
    """
    # mass matrix diagonal
    B = np.array([[m1, 0.0], [0.0, m2]])
    # gravity generalized force
    G = np.array([0.0, -m2 * g_val])
    tau = B @ qdd + G
    return tau

# Cost function (integral of squared generalized forces + penalties)
def energy_cost(x):
    q = x.reshape(2, N)
    # enforce start/end prismatic positions
    w_start_end = 1e5
    pen_start_end = w_start_end * (np.linalg.norm(q[:, 0] - q0)**2 + np.linalg.norm(q[:, -1] - qf)**2)

    # final EE penalty
    w_end = 2e5
    p_final = fk_position_prismatic(q[:, -1])
    pen_ee = w_end * np.linalg.norm(p_final - p_target)**2

    qd = np.gradient(q, dt, axis=1)
    qdd = np.gradient(qd, dt, axis=1)

    tau = np.zeros((2, N))
    for i in range(N):
        tau[:, i] = dynamics_prismatic(q[:, i], qd[:, i], qdd[:, i])

    energy = np.trapz(np.sum(tau**2, axis=0), t)

    # smoothness on force derivative
    dtau = np.gradient(tau, dt, axis=1)
    smooth_penalty = 1.0 * np.trapz(np.sum(dtau**2, axis=0), t)

    reg = 1e-2 * np.trapz(np.sum(q**2, axis=0), t)

    return energy + pen_start_end + pen_ee + smooth_penalty + reg

# Run the optimization
x0 = q_ref.flatten()
lb = np.tile([-1.0, -0.5], N)
ub = np.tile([ 1.0,  0.5], N)
bounds = [(lb[i], ub[i]) for i in range(2*N)]

print("Running optimization (this may take a moment)...")
res = minimize(
    energy_cost,
    x0,
    bounds=bounds,
    method='SLSQP',
    options={'maxiter': 500, 'ftol': 1e-6, 'disp': True}
)

if not res.success:
    print("WARNING: optimizer did not converge:", res.message)

x_opt = res.x
q_opt = x_opt.reshape(2, N)
qd_opt = np.gradient(q_opt, dt, axis=1)
qdd_opt = np.gradient(qd_opt, dt, axis=1)

# compute generalized forces for ref and opt
def compute_forces(q_traj):
    qd_traj = np.gradient(q_traj, dt, axis=1)
    qdd_traj = np.gradient(qd_traj, dt, axis=1)
    tau = np.zeros((2, N))
    for i in range(N):
        tau[:, i] = dynamics_prismatic(q_traj[:, i], qd_traj[:, i], qdd_traj[:, i])
    return tau

tau_ref = compute_forces(q_ref)
tau_opt = compute_forces(q_opt)

# Energy accumulation
E_ref = np.cumsum(np.sum(tau_ref**2, axis=0)) * dt
E_opt = np.cumsum(np.sum(tau_opt**2, axis=0)) * dt

print(f"Baseline trajectory energy: {E_ref[-1]:.6f}")
print(f"Optimized trajectory energy: {E_opt[-1]:.6f}")

# Final EE errors
p_ref_final = fk_position_prismatic(q_ref[:, -1])
p_opt_final = fk_position_prismatic(q_opt[:, -1])
print("Target EE position:", p_target)
print("Reference final EE pos:", p_ref_final, "error:", np.linalg.norm(p_ref_final - p_target))
print("Optimized final EE pos:", p_opt_final, "error:", np.linalg.norm(p_opt_final - p_target))

Running optimization (this may take a moment)...


/tmp/ipython-input-3780193496.py:75: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  energy = np.trapz(np.sum(tau**2, axis=0), t)
/tmp/ipython-input-3780193496.py:79: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  smooth_penalty = 1.0 * np.trapz(np.sum(dtau**2, axis=0), t)
/tmp/ipython-input-3780193496.py:81: DeprecationWarning: `trapz` is deprecated. Use `trapezoid` instead, or one of the numerical integration functions in `scipy.integrate`.
  reg = 1e-2 * np.trapz(np.sum(q**2, axis=0), t)


Optimization terminated successfully    (Exit mode 0)
            Current function value: 717.6157268533858
            Iterations: 213
            Function evaluations: 43512
            Gradient evaluations: 213
Baseline trajectory energy: 777.665455
Optimized trajectory energy: 712.651071
Target EE position: [0.108 0.118]
Reference final EE pos: [0.108 0.118] error: 0.0
Optimized final EE pos: [0.108 0.118] error: 4.501500623041109e-05


In [ ]:
# Create model and data
mj_model = mujoco.MjModel.from_xml_string(xml)
mj_data  = mujoco.MjData(mj_model)
renderer = mujoco.Renderer(mj_model, height=480, width=640)

scene_option = mujoco.MjvOption()
scene_option.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

duration = T
framerate = 60
frames = []

mujoco.mj_resetData(mj_model, mj_data)

# Start pose
start_j1_deg = -10
start_j2_deg = 20
mj_data.qpos[:2] = np.radians([start_j1_deg, start_j2_deg])

# Target pose
target_j1_deg = 80
target_j2_deg = 20
target_q = np.radians([target_j1_deg, target_j2_deg])
target_qd = np.array([0.0, 0.0])

# interpolation from start to target
def min_jerk(a):
    return 10*a**3 - 15*a**4 + 6*a**5

N = q_opt.shape[1]
q_smooth  = np.zeros_like(q_opt)
qd_smooth = np.zeros_like(q_opt)

q0 = np.radians([start_j1_deg, start_j2_deg])

for i in range(N):
    a = i / (N - 1)
    s = min_jerk(a)

    q_smooth[:, i] = q0 + s * (target_q - q0)

    # analytic derivative of min-jerk
    ds = (30*a**2 - 60*a**3 + 30*a**4) / duration
    qd_smooth[:, i] = ds * (target_q - q0)

# Ensure perfect landing at final point with zero jerk
q_smooth[:, -1]  = target_q
qd_smooth[:, -1] = target_qd

# Controller gains (reduced for smoothness)
kp = np.array([25.0, 25.0])
kd = np.array([ 3.0,  3.0])
ki = np.array([ 0.15, 0.15])

integ_err = np.zeros(2)

# Control loop
while mj_data.time < duration:

    t = mj_data.time
    idx = int((t / duration) * (N - 1))
    idx = np.clip(idx, 0, N - 1)

    q_des  = q_smooth[:, idx]
    qd_des = qd_smooth[:, idx]

    e    = q_des  - mj_data.qpos[:2]
    edot = qd_des - mj_data.qvel[:2]
    integ_err += e * mj_model.opt.timestep

    # Inverse dynamics feedforward
    mjtmp = np.zeros(mj_model.nv)
    mujoco.mj_rne(mj_model, mj_data, 0, mjtmp)
    tau_ff = mjtmp[:2]

    # Smooth PID control
    tau = tau_ff + kp*e + kd*edot + ki*integ_err
    mj_data.ctrl[:] = tau

    mujoco.mj_step(mj_model, mj_data)

    # Frame saving
    if len(frames) < mj_data.time * framerate:
        renderer.update_scene(mj_data, scene_option=scene_option)
        frames.append(renderer.render())

media.show_video(frames, fps=framerate)

This browser does not support the video tag.
